In [1]:
dataset_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno"
yaml_path = "/home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/train.yaml"

model_path = "/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/process_data/yolov8l_indoorvg4.pt"

import os
from ultralytics import YOLO

# load model
model = YOLO(model_path)

from ultralytics.cfg import get_cfg

cfg = get_cfg()

# load train data
# load yaml
from ultralytics.utils import yaml_load
from ultralytics.data import build_dataloader, build_yolo_dataset

data = yaml_load(yaml_path)

# load dataset
splits = [data['train'], data['val'], data['test']]

split = splits[0]
img_dir = os.path.join(split, 'images')
dataset = build_yolo_dataset(cfg, img_dir, 1, data)

/home/maelic/miniconda3/envs/phd/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load train data
# load yaml
from ultralytics.utils import yaml_load
from ultralytics.data import build_dataloader, build_yolo_dataset

data = yaml_load(yaml_path)

# load dataset
splits = [data['train'], data['val'], data['test']]

split = splits[0]
img_dir = os.path.join(split, 'images')
dataset = build_yolo_dataset(cfg, img_dir, 1, data)

print(dataset)


train: Scanning /home/maelic/Documents/PhD/MyModel/PhD_Commonsense_Enrichment/VG_refinement/data_tools/IndoorVG_4/YOLO_anno/train/labels.cache... 11739 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11739/11739 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


# Utils

In [4]:
new_dataset_path = dataset_path.strip().replace('YOLO_anno', 'YOLO_anno_2')
# create dir
os.makedirs(new_dataset_path, exist_ok=True)

In [6]:
from utils import bbox_iou
from tqdm import tqdm

iou_thres = 0.5
conf_thres = 0.5

dest_anno_path = new_dataset_path + '/train/labels'
# mkdir
if not os.path.exists(dest_anno_path):
    os.makedirs(dest_anno_path)

num_total = 0

# run model on dataset
for i, img in enumerate(tqdm(dataset)):
    # get img name
    img_name = img['im_file'].split('/')[-1].split('.')[0]
    # get anno file
    anno_file = dataset.label_files[i]
    # read anno file
    with open(anno_file, 'r') as f:
        anno = f.readlines()
    # get gt boxes
    gt_boxes = img['bboxes'].cuda()

    # forward pass
    results = model(img['im_file'], verbose=False)

    # iou on gt boxes and predictions
    for i, box in enumerate(results[0].boxes):
        iou = bbox_iou(box.xyxy, gt_boxes, xywh=True)

        if all(iou < iou_thres):
            # the box is a candidate for a new object
            if box.conf > conf_thres:
                # add to anno
                bbox = box.xyxyn[0]
                anno.append(f"{int(box.cls.item())} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
                num_total += 1

    # write out to dest_anno_path
    with open(dest_anno_path + f'/{img_name}.txt', 'w') as f:
        f.writelines(anno)

print(f"Total new objects added: {num_total}")

100%|██████████| 11739/11739 [08:18<00:00, 23.56it/s]

Total new objects added: 41194


In [ ]:
# display overall number of objects
num_objects = 0
for anno_file in os.listdir(dest_anno_path):
    with open(dest_anno_path + f'/{anno_file}', 'r') as f:
        anno = f.readlines()
        num_objects += len(anno)

print(f"Total number of objects: {num_objects}")